In [ ]:
import requests
import chardet
from bs4 import BeautifulSoup
import facebook
import time
import re

cfg = {
 "page_id" : "328183278116459", # Step 1
 "access_token" : input('Access_token: ') #step3
}

graph = facebook.GraphAPI(cfg['access_token'])

###############Global setting end###############

def getrawlist():
    
    r = requests.get("http://bcc.hlis.hlc.edu.tw/files/40-1000-12-1.php")
    r.encoding = 'utf-8'
    #print(r.text)

    soup = BeautifulSoup(r.text,"html.parser")
    #print(soup.text)

    sel = soup.select("table.baseTB.listTB.list_TABLE.hasBD.hasTH tbody tr")
    #print(sel)
    
    ret = []

    for s in sel:
    #s = sel[0]
        #for begin
        data = {}
        
        #abstract title & url
        atag = s.select("a")
        data["title"] = atag[0]["title"].strip()
        data["url"] = atag[0]["href"].strip()
        
        #abstract issuer
        
        data["issuer"]  = s.select("td")[1].text.strip()
        
        #abstract date
        
        data["date"]  = s.select("td")[2].text.strip()
        
        ret.append(data)
        
        #for end
    
    return ret

def filt(dataset):
    
    #print(dataset)
    
    #get latest post on facebook page
    r_api = graph.get_object(id=cfg["page_id"],fields="posts.limit(10)")
    #print(r_api)
    
    #find the latest post title
    for i in range(10):
        try:
            last_title = r_api['posts']['data'][i]['message'].strip()
            #print(last_title)
        except KeyError:
            last_title = ""
            
        if last_title != "":
            break
    
    #abstract title
    regex = re.compile(r'[^\/]*\/\s*(.*)\s+\/\s\d{4}\-\d{2}\-\d{2}') #match title
    match = regex.search(last_title)
    #print(type(match))
    
    if match is None:
        last_title = ""
    else:
        last_title = match[1]
    
    #print(last_title)
    
    #search title position
    for index,data in enumerate(dataset):
        if last_title != "" and last_title in data["title"]:
            #print("position:",index)

            #remove all datas after latest posted title(include it) *they're already posted
            for i in range(index,len(dataset)):
                dataset.pop()
                #print("delete:",dataset[i])
    
    #print(dataset)
    
    #make it ready for post
    ret = []
    for data in dataset:
        
        #print(data"\n")        
        
        ret.append({
            "message":'#'+data['issuer']+' / '+data['title']+' / '+data['date'],
            "link":data["url"]
        })
    
    return ret

def publisher(postset):
    
    r_api = []
    postset.reverse()
    for post in postset:
        
        #post text,link
        r = graph.put_object(cfg['page_id'],"feed",message=post["message"],link=post["link"])
        print("result : ",r)
        r_api.append(r)
        
        time.sleep(20)
        
    return r_api

#main

while True:
    
    now = time.localtime(time.time())
    
    print(now.tm_hour," : ",end="")
    
    if now.tm_hour >= 6 and now.tm_hour <= 20:
        
        print("execute")
        
        rawlist = getrawlist()
        #print(rawlist)
        
        ready4post = filt(rawlist)
        print(ready4post)
        
        result = publisher(ready4post)
        #print("result:",result)
        
    else:
        
        print("sleep")
        time.sleep(3600)
    
    time.sleep(600) #run one per ten minutes

Access_token: EAAHcEuS4fYoBAAuZAhdSj7CLb3YemP9ZBoLxo6AIrlzUMBVnON7iolyovFos0LJhGY6aZAmPc8PefoiI0meoyl8P0USAZCRialaWs01CvKEPnSsHj0WujfTbidvKu6Qj8foi3IdUekZCnRQnUjSWTDc1NyOvzVYZBDi7TVFg8Tt2X1LWUAyiBY
1  : sleep
2  : sleep
3  : sleep
4  : sleep
6  : execute
[{'message': '#學務處 / 賀！「才」高八斗，「藝」舉成名－108年度花工學生才藝發表會表演影片！！ / 2019-06-20', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17384,r12-1.php'}, {'message': '#教官室 / 本校108學年第1學期申請住宿事宜 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17383,r12-1.php'}, {'message': '#員生消費合作社 / 員生社第31屆理監事選舉訂於6月28日星期五早上,請大家踴躍參與投票 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17382,r12-1.php'}, {'message': '#輔導室 / 2019安生立命學生營隊 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17381,r12-1.php'}, {'message': '#輔導室 / 2019實踐家偏鄉青少年Money&You 654期公益營 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17379,r12-1.php'}, {'message': '#輔導室 / 重要訊息:通報不得超過24小時之規定 / 2019-06-19', 'link': 'http://bcc.hlis.hlc.edu.

result :  {'id': '328183278116459_332243104377143'}
result :  {'id': '328183278116459_332243277710459'}
result :  {'id': '328183278116459_332243507710436'}
16  : execute
[]
16  : execute
[{'message': '#教務處 / 107學年度第二學期期末考監考表 / 2019-06-20', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17397,r12-1.php'}]
result :  {'id': '328183278116459_332254197709367'}
16  : execute
[]
16  : execute
[]
16  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[{'message': '#學務處 / 107學年度整潔及生活榮譽競賽總成績排名及暑期返校打掃時間 / 2019-06-20', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17399,r12-1.php'}]
result :  {'id': '328183278116459_332303244371129'}
18  : execute
[]
18  : execute
[]
18  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute

13  : execute
[]
14  : execute
[]
14  : execute
[]
14  : execute
[{'message': '#實習處 / 轉知由東海大學舉辦「2019年青年程式設計競賽」 / 2019-06-24', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17421,r12-1.php'}]
result :  {'id': '328183278116459_334845997450187'}
14  : execute
[]
14  : execute
[]
14  : execute
[]
15  : execute
[]
15  : execute
[]
15  : execute
[]
15  : execute
[{'message': '#實習處 / 高二同學注意！108學年度就業導向專班計畫已通過，共3班60人，專班名稱及校外實習時間如說明。 / 2019-06-24', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17417,r12-1.php'}]
result :  {'id': '328183278116459_334877950780325'}
15  : execute
[]
15  : execute
[{'message': '#教務處 / 國立花蓮高工學生108學年度第一學期高一學生(升高二)轉科申請 / 2019-06-24', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17424,r12-1.php'}]
result :  {'id': '328183278116459_334887977445989'}
16  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
18  : ex

result :  {'id': '328183278116459_335413030726817'}
result :  {'id': '328183278116459_335413300726790'}
result :  {'id': '328183278116459_335413450726775'}
9  : execute
[{'message': '#教務處 / 107學年度第二學期期末考成績登錄通知 / 2019-06-25', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17431,r12-1.php'}]
result :  {'id': '328183278116459_335417917392995'}
9  : execute
[]
9  : execute
[{'message': '#人事室 / 公告本校108學年度第一學期第1次代理教師甄選簡章 / 2019-06-25', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17432,r12-1.php'}]
result :  {'id': '328183278116459_335427370725383'}
10  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
12  : execute
[{'message': '#人事室 / 訂於108年6月28日(五)中午於本校交誼廳辦理退休餐會，請同仁當日中午至交誼廳用餐 / 2019-06-25', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17433,r12-1.php'}]
result :  {'id': '328183278116459_335486277386159'}
12  : execute
[]
12  : e

15  : execute
[]
15  : execute
[]
15  : execute
[]
15  : execute
[]
15  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
16  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
17  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
21  : sleep
22  : sleep
23  : sleep
0  : sleep
1  : sleep
2  : sleep
4  : sleep
5  : sleep
6  : execute
[]
6  : execute
[]
6  : execute
[]
6  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
8  : execute
[]
8  : execute
[{'message': '#員生消費合作社 / 6月28日員生社第31屆理監事選舉請踴躍大家投票 / 2019-06-28', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17460,r12-

...

result :  {'id': '328183278116459_339509526983834'}
result :  {'id': '328183278116459_339509726983814'}
result :  {'id': '328183278116459_339509976983789'}
result :  {'id': '328183278116459_339510153650438'}
result :  {'id': '328183278116459_339510460317074'}
result :  {'id': '328183278116459_339510620317058'}
result :  {'id': '328183278116459_339510806983706'}
result :  {'id': '328183278116459_339510980317022'}
result :  {'id': '328183278116459_339511166983670'}
result :  {'id': '328183278116459_339511383650315'}
result :  {'id': '328183278116459_339511620316958'}
result :  {'id': '328183278116459_339511746983612'}
result :  {'id': '328183278116459_339511880316932'}
result :  {'id': '328183278116459_339512093650244'}
result :  {'id': '328183278116459_339512243650229'}
result :  {'id': '328183278116459_339512456983541'}
result :  {'id': '328183278116459_339512670316853'}
result :  {'id': '328183278116459_339512813650172'}
result :  {'id': '328183278116459_339512940316826'}
result :  {'

result :  {'id': '328183278116459_340572336877553'}
result :  {'id': '328183278116459_340572553544198'}
result :  {'id': '328183278116459_340572773544176'}
result :  {'id': '328183278116459_340572966877490'}
result :  {'id': '328183278116459_340573156877471'}
result :  {'id': '328183278116459_340573306877456'}
result :  {'id': '328183278116459_340573473544106'}
result :  {'id': '328183278116459_340573613544092'}
result :  {'id': '328183278116459_340573736877413'}
result :  {'id': '328183278116459_340573900210730'}
result :  {'id': '328183278116459_340574066877380'}
result :  {'id': '328183278116459_340574226877364'}
result :  {'id': '328183278116459_340574376877349'}
result :  {'id': '328183278116459_340574550210665'}
result :  {'id': '328183278116459_340574693543984'}
result :  {'id': '328183278116459_340574800210640'}
result :  {'id': '328183278116459_340574990210621'}
result :  {'id': '328183278116459_340575183543935'}
result :  {'id': '328183278116459_340575286877258'}
result :  {'

[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
21  : sleep
22  : sleep
23  : sleep
0  : sleep
1  : sleep
2  : sleep
4  : sleep
5  : sleep
6  : execute
[]
6  : execute
[]
6  : execute
[]
6  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
7  : execute
[]
8  : execute
[]
8  : execute
[]
8  : execute
[]
8  : execute
[]
8  : execute
[]
8  : execute
[]
9  : execute
[]
9  : execute
[]
9  : execute
[]
9  : execute
[]
9  : execute
[]
9  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
10  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
11  : execute
[]
12  : execute
[]
12  : execute
[]
12  : execute
[]
12  : execute
[]
12  : execute
[]
12  : execute
[]
13  : execute
[]
13  : execute
[]
13  : execute
[]
13  : execute
[]
13  : execute
[]
13  : execute
[]
14  : execute
[]
14  : execute
[]
14  : execute
[]
14  : execute
[]
14  : exec

[]
9  : execute
[]
9  : execute
[]
9  : execute
[]
9  : execute
[{'message': '#教務處 / 108學年度本校適性輔導安置錄取新生報到注意事項 / 2019-07-09', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17546,r12-1.php'}]
result :  {'id': '328183278116459_344111159857004'}
9  : execute
[]
10  : execute
[{'message': '#總務處 / 107-1學期行政會議及107學年度校務會議紀錄 / 2019-07-09', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-16369,r12-1.php'}]
result :  {'id': '328183278116459_344119233189530'}
10  : execute
[{'message': '#教務處 / 108學年度本校適性輔導安置錄取新生報到注意事項 / 2019-07-09', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17546,r12-1.php'}, {'message': '#輔導室 / 公告：函轉內政部針對身心障礙證明全面換證資訊 / 2019-07-08', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17545,r12-1.php'}, {'message': '#輔導室 / 公告：本校108學年度『資源班輔導員』應試名單 / 2019-07-08', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17544,r12-1.php'}, {'message': '#輔導室 / 公告：108年7-9月教育廣播電台『特別的愛』節目表 / 2019-07-08', 'link': 'http://bcc.hlis.hlc.edu.tw/files/14-1000-17543,r12-1.php'}, {'message': 

result :  {'id': '328183278116459_344280203173433'}
result :  {'id': '328183278116459_344280323173421'}
17  : execute
[{'message': '#學務處 / 第18期新世紀領導人才培育營/北一區初階/輔導老師影片(吳詠恩)(另開新視窗) / 2019-07-09', 'link': 'https://www.youtube.com/watch?v=FVGxj0Lz3LE'}, {'message': '#學務處 / 第18期新世紀領導人才培育營/北一區初階/回饋學員影片(薛旭宏)(另開新視窗) / 2019-07-09', 'link': 'https://youtu.be/_Cu2Y8NlrUc'}]
result :  {'id': '328183278116459_344284776506309'}
result :  {'id': '328183278116459_344284886506298'}
17  : execute
[{'message': '#學務處 / 第18期新世紀領導人才培育營/北一區初階/回饋學員影片(薛旭宏)(另開新視窗) / 2019-07-09', 'link': 'https://youtu.be/_Cu2y8NlrUc'}]
result :  {'id': '328183278116459_344289576505829'}
18  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
18  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
19  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
20  : execute
[]
21  : sleep
22  : sleep
23  : sleep
0  : 